## Rails 8: Comparing `permit` vs `expect` for Strong Parameters


#### Understand the differences, advantages, and proper usage of `permit` and `expect`
in Rails 8 parameter handling.

This touches on a subtle but important difference between `parameter whitelisting` and `parameter expectation` in Rails.

🧑‍💻 Technical / Developer-to-Developer

What makes `params.expect` a better choice than using `params.require(...).permit(...)` in this case?

Examples:

_________________________________________________________________________________________________
🥇 __1.__ 
`params.expect(user: [:email, :password, :password_confirmation])`

This is the preferred (modern) approach in Rails 7.1+.

✅ __What it does__:

- Declares exactly which parameters are expected (not just permitted).

- Rejects requests that don’t include those parameters, helping catch malformed input early.

- Makes the controller more predictable — it fails fast when something unexpected or missing happens.

💡__Key advantages__:

- Stronger validation at the boundary — if a required key is missing, Rails will raise an informative error (`ActionController::ParameterMissing`).

- Safer by default — you get explicit parameter structure checking (`useful for APIs`).

- Declarative and concise — reads as “__I expect these parameters__”, not just “__I allow these if they’re there__.”
_________________________________________________________________________________________________
🧾 __2.__
`params.require(:user).permit(:email, :password, :password_confirmation)`

This is the older strong parameters style (still valid and very common).

⚙️ __What it does__:

- Requires the top-level `:user` key to exist.

- Permits listed attributes if present — __but doesn’t complain if some are missing__.

- Commonly used in `create` and `update` actions for models.

⚠️ __Limitations__:

Doesn’t enforce presence of each field.

Allows extra flexibility — which can sometimes mean unexpected or partial input slips through silently.

You’ll need separate model-level validations to ensure all required fields exist.
______________________________________________________________________________________________________

🧠 Summary
| Feature                        | `expect`                       | `require(...).permit(...)`   |
| ------------------------------ | ------------------------------ | ---------------------------- |
| Available since                | Rails 7.1                      | Rails 4+                     |
| Validates *structure*          | ✅ Yes                          | ⚠️ Only top-level key        |
| Raises error on missing params | ✅ Yes                          | ⚠️ Only if top key missing   |
| Common use                     | APIs, modern apps              | Traditional MVC apps         |
| Intent                         | Declarative (“I expect these”) | Permissive (“I allow these”) |


#### This notebook needs __actionpack__ and __Rails 8__.

-----------------------------------------------------------------------

#### How to install `actionpack`:

#### Option 1 — Run from your terminal (recommended)

Exit the notebook and install it via your shell:
```
gem install actionpack
```

Then reopen your IRuby notebook.
This ensures the gem is installed in the same Ruby version managed by asdf or rbenv.

You can confirm inside IRuby:
```
require "action_controller"
puts "Action Pack version: #{ActionPack::VERSION::STRING}"
```

#### Option 2 — Install from inside the notebook:

In [1]:
# Load ActionController (part of ActionPack)
begin
  require "action_controller"
rescue LoadError
  puts "Installing actionpack..."
  system("gem install actionpack")
  Gem.clear_paths
  require "action_controller"
end

require "action_pack/version"
puts "✅ Action Pack version: #{ActionPack::VERSION::STRING}"


✅ Action Pack version: 8.0.3


(*) Below is a step-by-step guide to installing Rails 8 👇

In [2]:
require "rails"
Rails.version

"8.0.3"

-----------------------------------------------------------------------

## Let's Get Started!


| Method | Behavior | Structure Enforcement | Error on Missing Key |
|---------|------------|------------------------|-----------------------|
| `permit` | Whitelists allowed keys | ❌ No | ⚠️ Only top-level |
| `expect` | Validates presence and shape | ✅ Yes | ✅ Always |


In [3]:
require "action_controller"

# -------------------------------
# 1️⃣ Create a nested parameters object
# -------------------------------
params = ActionController::Parameters.new(
  user: { 
      posts: [
          { id: 1, title: "Hello", secret: "nope" }, 
          { id: 2, title: "World", secret: "nope" }
      ] 
  }
)

# -------------------------------
# 2️⃣ Pretty-print the class of params
# -------------------------------
puts "Class of params:"
pp params.class

puts ""  # blank line

# -------------------------------
# 3️⃣ Pretty-print each top-level key/value
# -------------------------------
puts "Top-level parameters:"
params.each do |key, value|
  puts "Key: #{key.inspect}, Value: #{value.inspect}"
end

puts ""  # blank line

# -------------------------------
# 4️⃣ Pretty-print nested structure
# -------------------------------
puts "Nested user parameters:"
params[:user].each do |key, value|
  puts "Key: #{key.inspect}, Value: #{value.inspect}"
end

puts ""  # blank line

puts "Nested posts array:"
params[:user][:posts].each_with_index do |post, index|
    puts " "
    puts "Post ##{index + 1}: #{post.inspect}"
    puts "⛔ Permitted? #{post.permitted?}"
end

Class of params:
ActionController::Parameters

Top-level parameters:
Key: "user", Value: #<ActionController::Parameters {"posts" => [{"id" => 1, "title" => "Hello", "secret" => "nope"}, {"id" => 2, "title" => "World", "secret" => "nope"}]} permitted: false>

Nested user parameters:
Key: "posts", Value: [#<ActionController::Parameters {"id" => 1, "title" => "Hello", "secret" => "nope"} permitted: false>, #<ActionController::Parameters {"id" => 2, "title" => "World", "secret" => "nope"} permitted: false>]

Nested posts array:
 
Post #1: #<ActionController::Parameters {"id" => 1, "title" => "Hello", "secret" => "nope"} permitted: false>
⛔ Permitted? false
 
Post #2: #<ActionController::Parameters {"id" => 2, "title" => "World", "secret" => "nope"} permitted: false>
⛔ Permitted? false


[#<ActionController::Parameters {"id" => 1, "title" => "Hello", "secret" => "nope"} permitted: false>, #<ActionController::Parameters {"id" => 2, "title" => "World", "secret" => "nope"} permitted: false>]

In [4]:
# -------------------------------
# 5️⃣ Require user and fetch posts
# -------------------------------
posts = params.require(:user).fetch(:posts)

# -------------------------------
#  Permit only :id and :title for each post
# -------------------------------

permitted_posts = posts.map do |post|
  # Wrap each hash in ActionController::Parameters and permit specific keys
  post.permit(:id, :title)
end

# -------------------------------
# 6️⃣ Print permitted posts and check if permitted
# -------------------------------
puts "Permitted posts (each post separated):"
permitted_posts.each_with_index do |post, index|
  puts " "
  puts "Post ##{index + 1}: #{post.inspect}"
  puts "✅ Permitted? #{post.permitted?}"
  
end

puts " "

Permitted posts (each post separated):
 
Post #1: #<ActionController::Parameters {"id" => 1, "title" => "Hello"} permitted: true>
✅ Permitted? true
 
Post #2: #<ActionController::Parameters {"id" => 2, "title" => "World"} permitted: true>
✅ Permitted? true
 


In [5]:
params

#<ActionController::Parameters {"user" => #<ActionController::Parameters {"posts" => [#<ActionController::Parameters {"id" => 1, "title" => "Hello", "secret" => "nope"} permitted: false>, #<ActionController::Parameters {"id" => 2, "title" => "World", "secret" => "nope"} permitted: false>]} permitted: false>} permitted: false>

That output is an __array__ — specifically, an array of __ActionController::Parameters__ objects.

__Explanation__

```
[
  #<ActionController::Parameters {"id"=>1} permitted: true>,
  #<ActionController::Parameters {"id"=>2} permitted: true>
]
```
The square brackets [ ... ] mean it’s a Ruby Array.

Inside the array, you have two elements — each one is an instance of ActionController::Parameters.

Each ActionController::Parameters object represents a permitted hash:
```
{"id"=>1}
{"id"=>2}
```

__Then__:

- params.require(:posts) returns an array of ActionController::Parameters objects (not hashes).

- You can directly call .permit(:id) on each.

- This works in __Rails 7+ and Rails 8+__, keeping your notebook future-proof.




#### `.to_h`
This is a very practical method when dealing with ActionController::Parameters.

Let’s dig into what .to_h actually does 👇

🧩 __Definition__

In Rails, `ActionController::Parameters#to_h`

➡️ converts a Parameters object into a plain Ruby hash.

🧠 Example:

In [6]:
params = ActionController::Parameters.new(id: 1, name: "Alice")
permitted = params.permit(:id, :name)
permitted.to_h


{"id" => 1, "name" => "Alice"}

✅ Summary
| Method        | Purpose                                                   | Returns                  |
| ------------- | --------------------------------------------------------- | ------------------------ |
| `permit`      | Marks parameters as safe to use                           | Same `Parameters` object |
| `to_h`        | Converts permitted parameters into a plain Ruby hash      | `Hash`                   |
| `map(&:to_h)` | Converts an array of `Parameters` objects to plain hashes | `Array<Hash>`            |


-----------------------------------------------------------------------

---------------------------------------------------------------------

# Hacking

---------------------------------------------------------------------

Below is a concise, __hands‑on demonstration__ showing that __improper parameter whitelisting__ — for example using `permit!` or `to_unsafe_h` — allows attacker-supplied keys (like `secret` or `admin`) to slip through and be used. Using explicit per-field `permit` or `params.expect` prevents this. 

We’ll use your exact params structure and show __unsafe__ vs. __safe__ handling with brief explanations and sample output.

Here are two IRuby/Jupyter cells:

 - __Cell A__ (unsafe) demonstrates how using `to_unsafe_h` or `permit!` __exposes attacker-supplied keys__.

- __Cell B__ (safe) shows explicit `per-post permitting` (and `expect`) to __drop unexpected keys__.



### Cell A — Unsafe handling (demonstration)

Run Cell A first to see the vulnerable outputs,then Cell B to see the safe approach.

In [7]:
# ===============================
# Cell A — Unsafe handling demo
# ===============================
require "action_controller"
require "pp"

# -------------------------------
# 0️⃣ Build attacker-supplied params
# -------------------------------
params = ActionController::Parameters.new(
  user: {
    posts: [
      { id: 1, title: "Hello", secret: "exfiltrate" },
      { id: 2, title: "World", secret: "exfiltrate2" }
    ],
    # attacker added a top-level sensitive flag
    admin: true
  }
)

# -------------------------------
# 1️⃣ Inspect original params
# -------------------------------
puts "Original params (class):"
pp params.class
puts ""
puts "Original params (raw):"
pp params
puts ""

# -------------------------------
# 2️⃣ UNSAFE: convert entire params to a plain hash
# - to_unsafe_h bypasses strong parameters protection
# -------------------------------
puts "UNSAFE: params.to_unsafe_h (everything exposed):"
unsafe_hash = params.to_unsafe_h
pp unsafe_hash
puts ""

# -------------------------------
# 3️⃣ UNSAFE: permit! on the :user key (permits all keys under user)
# - permit! marks everything as permitted under that subtree
# -------------------------------
puts "UNSAFE: params.require(:user).permit! (permits all under :user):"
permitted_everything = params.require(:user).permit!
pp permitted_everything
puts ""
puts "permitted_everything.permitted? => #{permitted_everything.permitted?}"
puts "permitted_everything.to_h =>"
pp permitted_everything.to_h
puts ""

# -------------------------------
# 4️⃣ Example consequence (illustrative)
# - If you did: current_user.update(permitted_everything.to_h)
#   the attacker-supplied :admin and each post's :secret would be applied
# -------------------------------
puts "CONSEQUENCE: attacker keys like :admin and post[:secret] are now available for mass-assignment."
puts ""


Original params (class):
ActionController::Parameters

Original params (raw):
#<ActionController::Parameters {"user" => {"posts" => [{"id" => 1, "title" => "Hello", "secret" => "exfiltrate"}, {"id" => 2, "title" => "World", "secret" => "exfiltrate2"}], "admin" => true}} permitted: false>

UNSAFE: params.to_unsafe_h (everything exposed):
{"user" =>
  {"posts" =>
    [{"id" => 1, "title" => "Hello", "secret" => "exfiltrate"},
     {"id" => 2, "title" => "World", "secret" => "exfiltrate2"}],
   "admin" => true}}

UNSAFE: params.require(:user).permit! (permits all under :user):
#<ActionController::Parameters {"posts" => [#<ActionController::Parameters {"id" => 1, "title" => "Hello", "secret" => "exfiltrate"} permitted: true>, #<ActionController::Parameters {"id" => 2, "title" => "World", "secret" => "exfiltrate2"} permitted: true>], "admin" => true} permitted: true>

permitted_everything.permitted? => true
permitted_everything.to_h =>
{"posts" =>
  [{"id" => 1, "title" => "Hello", "secret"

### Cell B — Safe handling
(explicit `permitting` + `expect`)

In [8]:
# ===============================
# Cell B — Safe handling demo
# ===============================
require "action_controller"
require "pp"

# -------------------------------
# 0️⃣ Recreate the same attacker-supplied params for a fair comparison
# -------------------------------
params = ActionController::Parameters.new(
  user: {
    posts: [
      { id: 1, title: "Hello", secret: "exfiltrate" },
      { id: 2, title: "World", secret: "exfiltrate2" }
    ],
    admin: true
  }
)

# -------------------------------
# 1️⃣ Require :user and fetch :posts (explicitly)
# -------------------------------
posts = params.require(:user).fetch(:posts)

puts "Posts extracted (raw):"
pp posts
puts ""

# -------------------------------
# 2️⃣ SAFE: Permit only allowed keys for each post (explicit per-element permit)
# - Wrap each post in ActionController::Parameters and permit only :id and :title
# - Any unexpected keys like :secret are dropped
# -------------------------------
permitted_posts = posts.map do |post|
  post.permit(:id, :title)
end

puts "SAFE: permitted_posts (Parameters objects):"
permitted_posts.each_with_index do |post, idx|
  puts "Post ##{idx + 1}: #{post.inspect}"
  puts "  Permitted? #{post.permitted?}"
end
puts ""

# -------------------------------
# 3️⃣ Convert permitted posts to plain Ruby hashes for serialization / assignment
# -------------------------------
plain_hashes = permitted_posts.map(&:to_h)

puts "SAFE: permitted_posts as plain hashes (secret dropped):"
plain_hashes.each_with_index do |h, idx|
  puts "Post ##{idx + 1}: #{h}"
end
puts ""

# -------------------------------
# 4️⃣ (Optional) Demonstrate params.expect (Rails 7.1+ / 8) to enforce shape
# - This validates the expected structure [ user: { posts: [ { id, title } ] } ]
# - It will raise if the shape is wrong (and would also exclude unexpected keys)
# -------------------------------
begin
  expected = params.expect(user: [posts: [[:id, :title]]])
  puts "params.expect validated structure and returned:"
  pp expected
  puts "params.expect(...).map(&:to_h) =>"
  pp expected.map(&:to_h)
rescue => e
  puts "params.expect raised: #{e.class}: #{e.message}"
end
puts ""

# -------------------------------
# 5️⃣ Summary printout
# -------------------------------
puts "SUMMARY:"
puts "- Unsafe approaches (to_unsafe_h / permit!) expose attacker-supplied keys like :admin and post[:secret]."
puts "- Explicit per-post permit(:id, :title) drops :secret (SAFE for mass-assignment)."
puts "- params.expect(...) enforces shape and provides additional protection for strict APIs."


Posts extracted (raw):
[#<ActionController::Parameters {"id" => 1, "title" => "Hello", "secret" => "exfiltrate"} permitted: false>,
 #<ActionController::Parameters {"id" => 2, "title" => "World", "secret" => "exfiltrate2"} permitted: false>]

SAFE: permitted_posts (Parameters objects):
Post #1: #<ActionController::Parameters {"id" => 1, "title" => "Hello"} permitted: true>
  Permitted? true
Post #2: #<ActionController::Parameters {"id" => 2, "title" => "World"} permitted: true>
  Permitted? true

SAFE: permitted_posts as plain hashes (secret dropped):
Post #1: {"id" => 1, "title" => "Hello"}
Post #2: {"id" => 2, "title" => "World"}

params.expect validated structure and returned:
#<ActionController::Parameters {"posts" => [#<ActionController::Parameters {"id" => 1, "title" => "Hello"} permitted: true>, #<ActionController::Parameters {"id" => 2, "title" => "World"} permitted: true>]} permitted: true>
params.expect(...).map(&:to_h) =>
params.expect raised: NoMethodError: undefined method

_________________________________________________________________________

#### Why `params.to_unsafe_h` is used in Cell A ?

It bypasses Strong Parameters.
`to_unsafe_h` returns a plain Ruby Hash containing every key the client sent — including keys you didn’t permit (e.g. `:secret`, `:admin`). __That’s the whole point__: it demonstrates what happens when you disable Rails’ parameter protections.

It makes attacker-supplied fields visible.
By printing the result of `to_unsafe_h`, you show that attacker-controlled values are now present in server-side data structures. This is important for a demo: we can visually prove that malicious keys reached the server.

__It shows the mass-assignment__ risk.
In many controllers the next step is to pass params into a model update/creation, e.g.

___________________________________________________________________________
Short runnable snippet to pair with your to_unsafe_h print (for clarity)

Add these lines after your unsafe_hash printing to make the consequences explicit:

In [9]:
# Show that the original params are unpermitted
puts "Original params permitted? => #{params.permitted?}"

# Show that to_unsafe_h contains sensitive keys (admin, secret)
puts "unsafe_hash[:user].keys => #{unsafe_hash[:user].keys.inspect}"

# Illustrative (do NOT run this in production) — what a naive mass-assignment would do:
puts "If you did: current_user.update(unsafe_hash[:user])"
puts "=> attacker keys like :admin or :secret would be applied to the model"


Original params permitted? => false
unsafe_hash[:user].keys => ["posts", "admin"]
If you did: current_user.update(unsafe_hash[:user])
=> attacker keys like :admin or :secret would be applied to the model


What this proves (short)

__1)__ `to_unsafe_h` and `permit!` are __dangerous__: they convert or mark everything as permitted, so any field the client sends (including secret, admin, role, etc.) becomes available to your server code. If your next step mass-assigns those parameters to models (e.g. `User.update(params[:user]))`, the attacker can change sensitive attributes.

__2)__ `permit` must be used explicitly and carefully: permitting only the exact keys you want for each nested item (e.g. `.permit(:id, :title))` drops unexpected keys like secret.

__3)__ `params.expect` (Rails 7.1+) provides stronger safety: it validates shape and presence and will raise if the structure is wrong — useful for strict APIs.

### Attack scenario (realistic sketch)

An attacker POSTs:

In [10]:
{
  "user": {
    "admin": true,
    "posts": [
      { "id": 1, "title": "Hello", "secret": "exfiltrate" }
    ]
  }
}


{user: {admin: true, posts: [{id: 1, title: "Hello", secret: "exfiltrate"}]}}

If your controller does: 

```
user.update(params.require(:user).permit!)
```
or
```
user.update(params.require(:user).to_h)
```

the admin flag and secret values may be applied to the model — resulting in privilege escalation or data leaks.

If instead you do:

```
posts = params.require(:user).fetch(:posts)
safe = posts.map { |p| ActionController::Parameters.new(p).permit(:id, :title) }
```

then admin and secret are not present in safe, and mass-assignment won't apply them.

Recommendations (practical)

Never use permit! or to_unsafe_h on user-supplied parameters in production.

Explicitly permit only the fields you expect, for each nested element.

Use params.expect on Rails 7.1+ to validate shape for strict APIs.

Validate again at the model-level (presence, allowed values) — defense in depth.

If you must accept arbitrary data, whitelist server-side what may be mass-assigned; keep secret/internal fields out of user-controlled hashes.

_____________________________________________________________________________________________________
### Bonus:
_____________________________________________________________________________________________________

#### RAILS 8 Instalation on Ubuntu
Step by step to install Rails 8 on Ubuntu Linux. I’ll assume you want it for Ruby 3.4+ (since Rails 8 requires a modern Ruby).

__Step 1__ — Update system packages

```
sudo apt update
sudo apt upgrade -y
sudo apt install -y curl gnupg build-essential libssl-dev libreadline-dev zlib1g-dev git
```
__Step 2__ — Install a Ruby version manager

It’s highly recommended to use rbenv or asdf so you can manage Ruby versions easily.

Using rbenv:
```
# Install rbenv and ruby-build
git clone https://github.com/rbenv/rbenv.git ~/.rbenv
cd ~/.rbenv && src/configure && make -C src
echo 'export PATH="$HOME/.rbenv/bin:$PATH"' >> ~/.bashrc
echo 'eval "$(rbenv init - bash)"' >> ~/.bashrc
source ~/.bashrc

git clone https://github.com/rbenv/ruby-build.git ~/.rbenv/plugins/ruby-build
```
__Step 3__ — Install Ruby 3.4+

```
rbenv install 3.4.4
rbenv global 3.4.4
ruby -v
# Output example: ruby 3.4.4
```

__Step 4__ — Install Node.js and Yarn (for Rails JS assets)

Rails 8 uses import maps / JS bundling, so you need Node:

```
curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
sudo apt install -y nodejs
node -v
# Example output: v20.x.x
```
Yarn is optional if you plan to use CSS/JS bundlers:

```
sudo npm install --global yarn
yarn -v
```
__Step 5__ — Install Rails 8

Once Ruby is installed, install Rails 8.x via gem:

```
gem install rails -v 8.0.3
rails -v
# Output: Rails 8.0.3
```
✅ You can replace 8.0.3 with the latest Rails 8 release.

__Step 6__ — Verify the installation

```
rails new my_app
cd my_app
bin/rails server
```


    Open your browser at http://localhost:3000

    You should see the Rails welcome page for Rails 8.


#####  -------------------------------------------------------------------------- THANK YOU! -------------------------------------------------------------------------